# Compare-11: compare NPI pair (from_npi, to_npi) count in 'undirected Hop teaming' to that in 'rootgraph (npi_team_rootgraph)' and check the overlap pair count.

Description: compare NPI pair (from_npi, to_npi) count in 'undirected Hop teaming' to that in 'rootgraph (npi_team_rootgraph)' and check the overlap pair count.

Starting Author: Amy Jin (amy@careset.com)

Date: July 27th, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) hop_db:                        Hop teaming database name
# 2）hop_table:                     Hop teaming table name
# 3) hop_npi:                       Hop teaming npi column
# 4) hop_npi_dest:                  Hop teaming npi_dest column

# 1) foia_db:                       FOIA PUF database name
# 2）foia_table:                    FOIA PUF table name
# 3) foia_npi:                      FOIA PUF from npi column
# 4) foia_npi_dest:                 FOIA PUF to npi column
# --------------------------------------- Outputs: --------------------------------------
# Test result:  
#    - Count of distinct (npi,npi_dest) pair in both files
#    - Count of distinct (npi,npi_dest) pair in hop teaming table
#    - Count of distinct (npi,npi_dest) pair in rootgraph table

def compare_11(hop_db, hop_table, hop_npi, hop_npi_dest, \
              rootgraph_db, rootgraph_table, rootgraph_npi, rootgraph_npi_dest):

    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # MySQL query to find distinct count of NPI pair in both Hop and rootgraph
        query = ('''
            SELECT COUNT(*)
            FROM {db}.{t} AS table1
            JOIN {db2}.{t2} AS table2
            ON table1.{npi} = table2.{npi2} 
                AND table1.{npi_dest} = table2.{npi_dest2} ;
        '''.format(db = hop_db, t = hop_table, \
                   npi = hop_npi, npi_dest = hop_npi_dest, \
                   db2 = rootgraph_db, t2 = rootgraph_table, \
                   npi2 = rootgraph_npi, npi_dest2 = rootgraph_npi_dest))

        cur.execute(query)
    
        print ('Count of distinct ({},{}) in both {}.{} and {}.{} is:'.format(hop_npi, hop_npi_dest, \
                                                                         hop_db, hop_table, \
                                                                         rootgraph_db, rootgraph_table))
        
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
         
        # MySQL query to find distinct count of NPI pair in Hop teaming 
        query = ('''
                SELECT COUNT(*)
                FROM (
                       SELECT *
                       FROM {db}.{t1} AS T1
                       GROUP BY T1.{npi}, T1.{npi_dest}
                     ) AS T2;
        '''.format(db = hop_db, t1 = hop_table, npi = hop_npi, npi_dest = hop_npi_dest))

        cur.execute(query)
    
        print ('Count of distinct ({},{}) pair in {}.{} is:'.format(hop_npi, hop_npi_dest, hop_db, hop_table))
        
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))

        # MySQL query to find distinct count of NPI pair in rootgraph
        query = ('''
                SELECT COUNT(*)
                FROM (
                       SELECT *
                       FROM {db}.{t1} AS T1
                       GROUP BY T1.{npi}, T1.{npi_dest}
                     ) AS T2;
        '''.format(db = rootgraph_db, t1 = rootgraph_table, npi = rootgraph_npi, npi_dest = rootgraph_npi_dest))

        cur.execute(query)
    
        print ('Count of distinct ({},{}) pair in {}.{} is:'.format(rootgraph_npi, rootgraph_npi_dest, rootgraph_db, rootgraph_table))
        
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            
        cur.close()
        connection.close()

## Test Example

In [4]:
compare_11('_amy', 'test_data_good','icd10cm', \
          'icd10cm', \
         '_amy', 'test_data_bad1','icd_dgns_cd',\
         'icd_dgns_cd')

Count of distinct (icd10cm,icd10cm) in both _amy.test_data_good and _amy.test_data_bad1 is:
0
Count of distinct (icd10cm,icd10cm) pair in _amy.test_data_good is:
2000
Count of distinct (icd_dgns_cd,icd_dgns_cd) pair in _amy.test_data_bad1 is:
138


## Internal Data Example

In [6]:
compare_11('npi_hop_RQ14', 'undirected', \
           'npi', 'npi_dest',\
          'npi_team_rootgraph_2014', \
           'undirected', 'npi', 'npi_dest')

Count of distinct (npi,npi_dest) in both npi_hop_RQ14.undirected and npi_team_rootgraph_2014.undirected is:
123033346
Count of distinct (npi,npi_dest) pair in npi_hop_RQ14.undirected is:
123034516
Count of distinct (npi,npi_dest) pair in npi_team_rootgraph_2014.undirected is:
159132078
